In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import glob
import os
from PIL import Image
import random
import tensorflow as tf
import time
from datetime import datetime

In [3]:
TRAIN_PATH = './data/train/'
LOGS_Path = "./logs"
BATCH_SIZE = 8

In [4]:
files_list = glob.glob(os.path.join(TRAIN_PATH,"**/*"))
def get_img_batch(files_list,batch_size=32,size=(224,224),should_normalise=True,should_noise=True):
   
    batch_cover = []
    batch_secret = []


    for i in range(batch_size):
        img_secret_path = random.choice(files_list)
        img_cover_path = random.choice(files_list)
        
        img_secret = np.array(Image.open(img_secret_path).resize(size).convert("RGB"),dtype=np.float32)
        img_cover = np.array(Image.open(img_cover_path).resize(size).convert("RGB"),dtype=np.float32)
        
        img_secret /= 255.
        img_cover /= 255.
        
        batch_cover.append(img_cover)
        batch_secret.append(img_secret)
        
    return np.array(batch_cover) , np.array(batch_secret)

In [5]:
def prep_network(secret_tensor):
    
    with tf.variable_scope('prep_net'):
        
        with tf.variable_scope("3x3_conv_branch"):
            conv_3x3 = tf.layers.conv2d(inputs=secret_tensor,filters=50,kernel_size=3,padding='same',name="1")
            conv_3x3 = tf.layers.conv2d(inputs=conv_3x3,filters=50,kernel_size=3,padding='same',name="2")
            conv_3x3 = tf.layers.conv2d(inputs=conv_3x3,filters=50,kernel_size=3,padding='same',name="3")
            conv_3x3 = tf.layers.conv2d(inputs=conv_3x3,filters=50,kernel_size=3,padding='same',name="4")
            
        with tf.variable_scope("4x4_conv_branch"):
            conv_4x4 = tf.layers.conv2d(inputs=secret_tensor,filters=50,kernel_size=4,padding='same',name="1")
            conv_4x4 = tf.layers.conv2d(inputs=conv_4x4,filters=50,kernel_size=4,padding='same',name="2")            
            conv_4x4 = tf.layers.conv2d(inputs=conv_4x4,filters=50,kernel_size=4,padding='same',name="3")
            conv_4x4 = tf.layers.conv2d(inputs=conv_4x4,filters=50,kernel_size=4,padding='same',name="4")

        with tf.variable_scope("5x5_conv_branch"):
            conv_5x5 = tf.layers.conv2d(inputs=secret_tensor,filters=50,kernel_size=5,padding='same',name="1")
            conv_5x5 = tf.layers.conv2d(inputs=conv_5x5,filters=50,kernel_size=5,padding='same',name="2")            
            conv_5x5 = tf.layers.conv2d(inputs=conv_5x5,filters=50,kernel_size=5,padding='same',name="3")
            conv_5x5 = tf.layers.conv2d(inputs=conv_5x5,filters=50,kernel_size=5,padding='same',name="4")
            
        concat_1 = tf.concat([conv_3x3,conv_4x4,conv_5x5],axis=3,name='concat_1')
        
        conv_5x5 = tf.layers.conv2d(inputs=concat_1,filters=50,kernel_size=5,padding='same',name="final_5x5")
        conv_4x4 = tf.layers.conv2d(inputs=concat_1,filters=50,kernel_size=5,padding='same',name="final_4x4")
        conv_3x3 = tf.layers.conv2d(inputs=concat_1,filters=50,kernel_size=5,padding='same',name="final_3x3")
        
        concat_final = tf.concat([conv_5x5,conv_4x4,conv_3x3],axis=3,name='concat_final')


        return concat_final

    
def hiding_network(cover_tensor,prep_output):
    
    with tf.variable_scope('hide_net'):
        concat_input = tf.concat([cover_tensor,prep_output],axis=3,name='images_features_concat')
        
        with tf.variable_scope("3x3_conv_branch"):
            conv_3x3 = tf.layers.conv2d(inputs=concat_input,filters=50,kernel_size=3,padding='same',name="1")
            conv_3x3 = tf.layers.conv2d(inputs=conv_3x3,filters=50,kernel_size=3,padding='same',name="2")
            conv_3x3 = tf.layers.conv2d(inputs=conv_3x3,filters=50,kernel_size=3,padding='same',name="3")
            conv_3x3 = tf.layers.conv2d(inputs=conv_3x3,filters=50,kernel_size=3,padding='same',name="4")
            
        with tf.variable_scope("4x4_conv_branch"):
            conv_4x4 = tf.layers.conv2d(inputs=concat_input,filters=50,kernel_size=4,padding='same',name="1")
            conv_4x4 = tf.layers.conv2d(inputs=conv_4x4,filters=50,kernel_size=4,padding='same',name="2")            
            conv_4x4 = tf.layers.conv2d(inputs=conv_4x4,filters=50,kernel_size=4,padding='same',name="3")
            conv_4x4 = tf.layers.conv2d(inputs=conv_4x4,filters=50,kernel_size=4,padding='same',name="4")

        with tf.variable_scope("5x5_conv_branch"):
            conv_5x5 = tf.layers.conv2d(inputs=concat_input,filters=50,kernel_size=5,padding='same',name="1")
            conv_5x5 = tf.layers.conv2d(inputs=conv_5x5,filters=50,kernel_size=5,padding='same',name="2")            
            conv_5x5 = tf.layers.conv2d(inputs=conv_5x5,filters=50,kernel_size=5,padding='same',name="3")
            conv_5x5 = tf.layers.conv2d(inputs=conv_5x5,filters=50,kernel_size=5,padding='same',name="4")
            
        concat_1 = tf.concat([conv_3x3,conv_4x4,conv_5x5],axis=3,name='concat_1')
        
        conv_5x5 = tf.layers.conv2d(inputs=concat_1,filters=50,kernel_size=5,padding='same',name="final_5x5")
        conv_4x4 = tf.layers.conv2d(inputs=concat_1,filters=50,kernel_size=5,padding='same',name="final_4x4")
        conv_3x3 = tf.layers.conv2d(inputs=concat_1,filters=50,kernel_size=5,padding='same',name="final_3x3")
        
        concat_final = tf.concat([conv_5x5,conv_4x4,conv_3x3],axis=3,name='concat_final')
    
        output = tf.layers.conv2d(inputs=concat_final,filters=3,kernel_size=3,padding='same',name='output')



        return output
    
        
        
def reveal_network(container_tensor):
    
    with tf.variable_scope('reveal_net'):
        
        with tf.variable_scope("3x3_conv_branch"):
            conv_3x3 = tf.layers.conv2d(inputs=container_tensor,filters=50,kernel_size=3,padding='same',name="1")
            conv_3x3 = tf.layers.conv2d(inputs=conv_3x3,filters=50,kernel_size=3,padding='same',name="2")
            conv_3x3 = tf.layers.conv2d(inputs=conv_3x3,filters=50,kernel_size=3,padding='same',name="3")
            conv_3x3 = tf.layers.conv2d(inputs=conv_3x3,filters=50,kernel_size=3,padding='same',name="4")
            
        with tf.variable_scope("4x4_conv_branch"):
            conv_4x4 = tf.layers.conv2d(inputs=container_tensor,filters=50,kernel_size=4,padding='same',name="1")
            conv_4x4 = tf.layers.conv2d(inputs=conv_4x4,filters=50,kernel_size=4,padding='same',name="2")            
            conv_4x4 = tf.layers.conv2d(inputs=conv_4x4,filters=50,kernel_size=4,padding='same',name="3")
            conv_4x4 = tf.layers.conv2d(inputs=conv_4x4,filters=50,kernel_size=4,padding='same',name="4")

        with tf.variable_scope("5x5_conv_branch"):
            conv_5x5 = tf.layers.conv2d(inputs=container_tensor,filters=50,kernel_size=5,padding='same',name="1")
            conv_5x5 = tf.layers.conv2d(inputs=conv_5x5,filters=50,kernel_size=5,padding='same',name="2")            
            conv_5x5 = tf.layers.conv2d(inputs=conv_5x5,filters=50,kernel_size=5,padding='same',name="3")
            conv_5x5 = tf.layers.conv2d(inputs=conv_5x5,filters=50,kernel_size=5,padding='same',name="4")
            
        concat_1 = tf.concat([conv_3x3,conv_4x4,conv_5x5],axis=3,name='concat_1')
        
        conv_5x5 = tf.layers.conv2d(inputs=concat_1,filters=50,kernel_size=5,padding='same',name="final_5x5")
        conv_4x4 = tf.layers.conv2d(inputs=concat_1,filters=50,kernel_size=5,padding='same',name="final_4x4")
        conv_3x3 = tf.layers.conv2d(inputs=concat_1,filters=50,kernel_size=5,padding='same',name="final_3x3")
        
        concat_final = tf.concat([conv_5x5,conv_4x4,conv_3x3],axis=3,name='concat_final')
    
    output = tf.layers.conv2d(inputs=concat_final,filters=3,kernel_size=3,padding='same',name='output')

    return output


def get_loss_op(secret_true,secret_pred,cover_true,cover_pred,beta=.25):
    
    secret_mse = tf.losses.mean_squared_error(secret_true,secret_pred)
    cover_mse = tf.losses.mean_squared_error(cover_true,cover_pred)
    final_loss = cover_mse + beta*secret_mse
    return final_loss

In [6]:
batch = get_img_batch(files_list=files_list,batch_size=1)

In [7]:


sess = tf.InteractiveSession(graph=tf.Graph())



merged_summary_op = tf.summary.merge_all()


secret_tensor = tf.placeholder(shape=[None,224,224,3],dtype=tf.float32,name="input_prep")
cover_tensor = tf.placeholder(shape=[None,224,224,3],dtype=tf.float32,name="input_hide")


prep_output_op = prep_network(secret_tensor)
hiding_output_op = hiding_network(cover_tensor=cover_tensor,prep_output=prep_output_op)
reveal_output_op = reveal_network(hiding_output_op)


loss_op = get_loss_op(secret_tensor,reveal_output_op,cover_tensor,hiding_output_op)



tf.summary.scalar('cost_value', loss_op)
tf.summary.image('input_secret',secret_tensor)
tf.summary.image('input_cover',cover_tensor)
tf.summary.image('output_cover',hiding_output_op)
tf.summary.image('reveal_output',reveal_output_op)




merged_summary_op = tf.summary.merge_all()



writer = tf.summary.FileWriter(f"logs/{datetime.now()}",sess.graph)
saver = tf.train.Saver()



In [8]:

minimize_op = tf.train.AdamOptimizer(0.0002).minimize(loss_op)
sess.run(tf.global_variables_initializer())



In [14]:
for ep in range(3000):
    
    
    for step in range(len(files_list)//BATCH_SIZE + 1):
        covers,secrets = get_img_batch(files_list=files_list,batch_size=BATCH_SIZE)

        _,loss = sess.run([minimize_op,loss_op],
                                  feed_dict={secret_tensor:secrets,cover_tensor:covers})
    
        if ep % 10 ==0 :
            print(loss)
            summary = sess.run(merged_summary_op,feed_dict={secret_tensor:secrets,cover_tensor:covers})
            writer.add_summary(summary,ep)
        
#     if ep % 300 ==0 : 
        save_path = saver.save(sess, f"./checkpoints/Epoch-{ep}--Loss{loss}.chpk")
        

    

0.10128
0.087528
0.07919
0.0731602
0.0715436
0.0774648


KeyboardInterrupt: 

In [ ]:
# sess.close()

# writer.close()

In [ ]:
covers,secrets = batch
cover = covers.squeeze()
secret = secrets.squeeze()
plt.imshow(cover)
plt.show()
plt.imshow(secret)
plt.show()


In [ ]:
test_placeholder = tf.placeholder(shape=[1,224,224,3],dtype=tf.float32)
with tf.variable_scope("",reuse=True) :
    reveal_output_test = reveal_network(test_placeholder)

batch2 = get_img_batch(files_list=files_list,batch_size=1) 
covers,secrets = batch2
cover = covers.squeeze()
secret = secrets.squeeze()

plt.imshow(cover)
plt.show()
plt.imshow(secret)
plt.show()


reveal_img,hiding_output = sess.run([reveal_output_op,hiding_output_op],feed_dict={cover_tensor:covers,secret_tensor:secrets})
# reveal_img.shape

In [ ]:
plt.imshow(np.clip(reveal_img.squeeze(),0,1))

In [ ]:
plt.imshow(np.clip(hiding_output.squeeze(),0,1))